In [1]:
import numpy as np
import pandas as pd
import scipy
import spacy
import sys
from collections import Counter
from time import time
import torch
from sklearn import model_selection
import math
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
# data = pd.read_csv('articles1.csv')

In [ ]:


# # Make a DataFrame with articles by our chosen authors
# # Include author names and article titles.

# # Make a list of the 10 chosen author names
# names = data.author.value_counts()[data.author.value_counts()>100][-30:].index.tolist()

# # DataFrame for articles of all chosen authors
# authors_data = pd.DataFrame()
# for name in names:
#     # Select each author's data
#     articles = data[data.author==name][:100][['title','content','author']]
#     # Append it to the DataFrame
#     authors_data = authors_data.append(articles)

# authors_data = authors_data.reset_index().drop('index',1)
    
# authors_data.head()

In [ ]:
# # Look for duplicates
# print('Number of articles:',authors_data.shape[0])
# print('Unique articles:',len(np.unique(authors_data.index)))

# # Number of authors
# print('Unique authors:',len(np.unique(authors_data.author)))
# print('')
# print('Articles by author:\n')

# # Articles counts by author
# print(authors_data.author.value_counts())

In [ ]:
# #!{sys.executable} -m spacy download en

# t0 = time()

# # Load spacy NLP object
# nlp = spacy.load("en_core_web_sm")
# # A list to store common words by all authors
# common_words = []

# # A dictionary to store the spacy_doc object of each author
# authors_docs = {}

# for name in names:
#     # Corpus is all the text written by that author
#     corpus = ""
#     # Grab all rows of current author, along the 'content' column
#     author_content = authors_data.loc[authors_data.author==name,'content']
    
#     # Merge all articles in to the author's corpus
#     for article in author_content:
#         corpus = corpus + article
#     # Let Spacy parse the author's body of text
#     doc = nlp(corpus)
    
#     # Store the doc in the dictionary
#     authors_docs[name] = doc
        
#     # Filter out punctuation and stop words.
#     lemmas = [token.lemma_ for token in doc
#                 if not token.is_punct and not token.is_stop]
        
#     # Return the most common words of that author's corpus.
#     bow = [item[0] for item in Counter(lemmas).most_common(1000)]
    
#     # Add them to the list of words by all authors.
#     for word in bow:
#         common_words.append(word)

# # Eliminate duplicates
# common_words = set(common_words)
    
# print('Total number of common words:',len(common_words))
# print("done in %0.3fs" % (time() - t0))

In [ ]:
# # Let's see our 10 authors in the dictionary
# lengths = []
# for k,v in authors_docs.items():
#     print(k,'corpus contains',len(v),' words.')
#     lengths.append(len(v))



In [ ]:
# # check for lower case words
# common_words = pd.Series(pd.DataFrame(columns=common_words).columns)
# print('Count of all common_words:',len(common_words))
# print('Count of lowercase common_words:',np.sum([word.islower() for word in common_words]))

# # Turn all common_words into lower case
# common_words = [word.lower() for word in common_words]
# print('Count of lowercase common_words (After Conversion):',np.sum([word.islower() for word in common_words]))


# # We must remove these in to avoid conflicts with existing features.
# if 'author' in common_words:
#     common_words.remove('author')
# if 'title' in common_words:
#     common_words.remove('title')
# if 'content' in common_words:
#     common_words.remove('content')

In [ ]:
# # Count the number of times a common_word appears in each article
# # (about 3Hrs processing)

# bow_counts = pd.DataFrame()
# for name in names:
#     # Select X articles of that author
#     articles = authors_data.loc[authors_data.author==name,:][:50]
#     bow_counts = bow_counts.append(articles)
# bow_counts = bow_counts.reset_index().drop('index',1)

# # Use common_words as the columns of a temporary DataFrame
# df = pd.DataFrame(columns=common_words)

# # Join BOW features with the author's content
# bow_counts = bow_counts.join(df)

# # Initialize rows with zeroes
# bow_counts = bow_counts.loc[:,~bow_counts.columns.duplicated()].copy()
# bow_counts.loc[:,common_words] = 0

# # Fill the DataFrame with counts of each feature in each article
# t0 = time()
# for i, article in enumerate(bow_counts.content):
#     doc = nlp(article)
#     for token in doc:
#         if token.lemma_.lower() in common_words:
#             bow_counts.loc[i,token.lemma_.lower()] += 1
#     # Print a message every X articles
#     if i % 50 == 0:
#         if time()-t0 < 3600: # if less than an hour in seconds
#             print("Article ",i," done after ",(time()-t0)/60,' minutes.')
#         else:
#             print("Article ",i," done after ",(time()-t0)/60/60,' hours.')

In [ ]:
# This saves the long-awaited data into a pickle file for easy recovery
#bow_counts.to_pickle('bow_counts')
# Read it back in with the following
bow_counts = pd.read_pickle('bow_counts')

In [ ]:
counter = 0
newy = np.zeros(len(y))
last = y[0]
for i in range(len(y)):
    if y[i] != last:
        counter+=1
    newy[i] = counter
    last = y[i]

In [ ]:
y = bow_counts['author']
X = bow_counts.drop(['content','author','title'], 1)
X = X.loc[:,~X.columns.duplicated()].copy()
X = X.to_numpy()
N, M = X.shape
import torch.nn as nn
#X = scipy.stats.zscore(X)

n_hidden_units = 2     # number of hidden units
n_replicates = 1        # number of networks trained in each k-fold
max_iter = 10000         # stop criterion 2 (max epochs in training)

# K-fold crossvalidation
#K = 3                   # only five folds to speed up this example
#CV = model_selection.KFold(K, shuffle=True)
model = torch.nn.Sequential(
                    torch.nn.Linear(M, n_hidden_units), #M features to H hiden units
                    torch.nn.Tanh(),   # 1st transfer function,
                    torch.nn.Linear(n_hidden_units, 1), # H hidden units to 1 output neuron
                    torch.nn.Sigmoid(), # final tranfer function
                    )
#loss_fn = torch.nn.MSELoss
# class Net(nn.Module):
#     def __init__(self,D_in,H,D_out):
#         super(Net,self).__init__()
#         self.linear1=nn.Linear(D_in,H)
#         self.linear2=nn.Linear(H,D_out)

        
#     def forward(self,x):
#         x=torch.sigmoid(self.linear1(x))  
#         x=self.linear2(x)
#         return x
# model=Net(M,25,30)
class Net(nn.Module):
    def __init__(self,D_in,H,H2,D_out):
        super(Net,self).__init__()
        self.linear1=nn.Linear(D_in,H)
        self.linear2=nn.Linear(H,H2)
        self.linear3=nn.Linear(H2,D_out)
        
    def forward(self,x):
        x=F.relu(self.linear1(x))
        x=F.relu(self.linear2(x))  
        x=self.linear3(x)
        return x
model=Net(M,120,50,30)
loss_fn = torch.nn.CrossEntropyLoss()

In [ ]:
x, x_val, y, y_val = train_test_split(X, newy, test_size=0.33, random_state=42)

In [ ]:
x_train = x.reshape(-1, x.shape[1]).astype('float32')
y_train = y

x_val = x_val.reshape(-1, x_val.shape[1]).astype('float32')
y_val = y_val

In [ ]:
x_val = torch.from_numpy(x_val)
y_val = torch.from_numpy(y_val)

In [ ]:
class Data(Dataset):
    def __init__(self):
        self.x=torch.from_numpy(x_train)
        self.y=torch.from_numpy(y_train)
        self.len=self.x.shape[0]
    def __getitem__(self,index):      
        return self.x[index], self.y[index]
    def __len__(self):
        return self.len
data_set=Data()

In [ ]:
trainloader=DataLoader(dataset=data_set,batch_size=64)

In [ ]:
learning_rate=0.1
optimizer=torch.optim.SGD(model.parameters(), lr=learning_rate)

In [ ]:
n_epochs=5000
loss_list=[]

#n_epochs
for epoch in range(n_epochs):
    for x, y in trainloader:
        #clear gradient 
        optimizer.zero_grad()
        #make a prediction 
        z=model(x)
        # calculate loss, da Cross Entropy benutzt wird muss ich in den loss Klassen vorhersagen, 
        # also Wahrscheinlichkeit pro Klasse. Das mach torch.max(y,1)[1])
        loss=loss_fn(z,y.long())
        # calculate gradients of parameters 
        loss.backward()
        # update parameters 
        optimizer.step()
        
        loss_list.append(loss.data)

In [ ]:
z=model(x_val)
yhat=torch.max(z.data,1)
errors = 0
for i in range(len(y_val)):
    if yhat[1][i] != y_val[i]:
        errors+=1
print(errors, 'errors out of', len(y_val), 'inputs giving us an error rate of', errors/len(y_val))

In [ ]:
for i in range(len(y_val)):
    print(int(yhat[1][i]), int(y_val[i]))

In [2]:


# Assuming that we are on a CUDA machine, this should print a CUDA device:

print(device)

cuda:0
